In [1]:
import pandas as pd
import ast
from funciones import extract
import pyarrow
import json
import gzip
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
%load_ext memory_profiler
import pyarrow.parquet as pq

In [3]:
item_user = pd.read_csv(r'data_set_limpio\item_cantidad_usuarios.csv.gz')

In [2]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [4]:
def userdata(user: str):
    try:
        usuario = 0
        chunks = pd.read_csv(r'data_set_limpio\item_desplegado.csv.gz', chunksize=200000)
        for chunk in chunks:
            if chunk['user'].isin([user]).any():
                usuario = chunk[chunk['user'] == user]
                break

        resultado = {
            'Usuario' : user,
            'Dinero gastado' : str(usuario['price'].sum())+'$',
            'cantidad de items' : str(item_user.loc[item_user['user_id'] == user]['items_count'].values[0]),
            'Porcentaje de recomendaciones' :str(round((len(opinion.loc[opinion['user'] == user])  / item_user.loc[item_user['user_id'] == user]['items_count'].values[0])*100,2))+'%'}
        
        return resultado
    except:
        return 'El usuario no se encuentra en la base de datos.'


In [5]:
userdata('76561198329548331')

{'Usuario': '76561198329548331',
 'Dinero gastado': '0.0$',
 'cantidad de items': '7',
 'Porcentaje de recomendaciones': '0.0%'}

usuarios probados - iteracion de chunks

indice  -   usuario
1000004     conordarley1998 -> peak memory: 320.53 MiB, increment: 193.39 MiB 4.42s
2000003     xenocide1310 -> peak memory: peak memory: 324.86 MiB 4.38 s
2050004     76561198073938058 -> peak memory: 323.74 MiB 5.1s
1050004     76561198076447620 -> peak memory: 325.99 MiB 5 s
550004      aromortis -> peak memory: 320.77 MiB 5 s

usuarios probados - iteracion de chunks optimizada

indice  -   usuario
1000004     conordarley1998 -> peak memory: 171.46 MiB 2.4s => 1.8 s
2000003     xenocide1310 -> peak memory: 140.34 MiB 3.9 s 100000 chunks = 2.9s
2050004     76561198073938058 -> peak memory: 140.45 MiB 4.0 s = 3
1050004     76561198076447620 -> peak memory: 140.23 MiB 2.3 s =  1.9
550004      aromortis -> peak memory: 138.73 MiB    = 1.3

In [19]:
item_user

,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137
3,Riot-Punch,328
4,doctr,541
...,...,...
71499,76561198320136420,321
71500,ArkPlays7,4
71501,76561198323066619,22
71502,76561198326700687,177
